In [1]:
!pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

import torch
print (torch.cuda.get_device_name(0))

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import sys
from torch.utils.data.dataset import Dataset




    100% |████████████████████████████████| 512.6MB 50.2MB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58bfe000 @  0x7f30db6382a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 4.1MB/s 
    100% |████████████████████████████████| 2.0MB 12.8MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
Tesla K80


In [2]:
batch_size = 100

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Processing...
Done!


In [0]:
class LogitDataset(Dataset):
    def __init__(self):
        # stuff
        self.data = []
        self.soft_targets = []
        self.hard_targets = []
        
    def __getitem__(self, index):
        # stuff
        return (self.data[index], self.soft_targets[index], self.hard_targets[index])

    def __len__(self):
        return len(self.data)
     
    def add(self, img, soft_label, hard_label):
        self.data.append(img)
        self.soft_targets.append(soft_label)
        self.hard_targets.append(hard_label)
        
        
    def stackAll(self):
        self.data = torch.stack(self.data).reshape(-1, 1, 28, 28)
        self.soft_targets = torch.stack(self.soft_targets).reshape(-1, 10)
        self.hard_targets = torch.stack(self.hard_targets).reshape(-1)

      
def cross_entropy(logsoftmax_pred, soft_targets, hard_pred, hard_targets, alpha):
    cross_entropy_soft = torch.mean(torch.sum(- soft_targets * logsoftmax_pred, 1))
    
    hard_cross_entropy_criterion = nn.CrossEntropyLoss()
    cross_entropy_hard = hard_cross_entropy_criterion(hard_pred, hard_targets)
    
    return torch.mul(cross_entropy_soft, alpha) + torch.mul(cross_entropy_hard, 1.0-alpha)
  


# Convolutional neural network (two convolutional layers)
class TeacherNet(nn.Module):
    def __init__(self, num_classes=10):
        super(TeacherNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        self.dropout_1 = nn.Dropout(p=0.25)
        self.dropout_2 = nn.Dropout(p=0.25)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.dropout_1(out)
        out = self.layer2(out)
        out = self.dropout_2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

class StudentNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(StudentNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size) 
        self.fc3 = nn.Linear(hidden_size, num_classes)  
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [4]:
# Hyper-parameters 
input_size = 784
hidden_size = 1200
num_classes = 10
num_epochs = 100
batch_size = 100
learning_rate = 0.001
train_temp = 4.0 
teacher_hidden = 1200
student_hidden = 20
weight_ratio = 0.9
CNN_TRAIN_EPOCH_LIMIT = 100

# Initialize teacher and student model
teacher_model = TeacherNet(num_classes).to(device)
student_vanila_model = StudentNet(input_size, student_hidden, num_classes).to(device)
student_enhanced_model = StudentNet(input_size, student_hidden, num_classes).to(device)

# Loss and optimizer for teacher
criterion_teacher = nn.CrossEntropyLoss()
optimizer_teacher = torch.optim.Adam(teacher_model.parameters(), lr=learning_rate)  

# Loss and optimizer for student vanila
criterion_student_vanila = nn.CrossEntropyLoss()
optimizer_student_vanila = torch.optim.Adam(student_vanila_model.parameters(), lr=learning_rate)  

# Train the CNN Teacher Model

# Train the vanila Student Model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images_for_teacher = images.to(device)
        images_for_student = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Teacher Network
        if epoch < CNN_TRAIN_EPOCH_LIMIT:
          outputs_teacher = teacher_model(images_for_teacher) # teacher softmax
          loss_teacher = criterion_teacher(outputs_teacher, labels)

          optimizer_teacher.zero_grad()
          loss_teacher.backward()
          optimizer_teacher.step()
        
        
        # Student Network
        outputs_student_vanila = student_vanila_model(images_for_student) # student softmax
        loss_student_vanila = criterion_student_vanila (outputs_student_vanila, labels)
        
        optimizer_student_vanila.zero_grad()
        loss_student_vanila.backward()
        optimizer_student_vanila.step()
        
        if (i+1) % 600 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss Teacher: {:.4f}, Loss Student: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss_teacher.item(), loss_student_vanila.item()))

# Test the model
teacher_model = teacher_model.eval()
student_vanila_model = student_vanila_model.eval()

with torch.no_grad():
    correct1, correct2 = 0, 0
    total = 0
    for images, labels in test_loader:
        
        images_for_teacher = images.to(device)
        images_for_student = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        output1, output2 = teacher_model(images_for_teacher), student_vanila_model(images_for_student)
        _, predicted1 = torch.max(output1.data, 1)
        _, predicted2 = torch.max(output2.data, 1)
        total += labels.size(0)
        correct1 += (predicted1 == labels).sum().item()
        correct2 += (predicted2 == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: TEACHER: {}, STUDENT: {} %'.format(100 * correct1 / total, 100 * correct2 / total))
    
print ("CREATING NEW DATASET WITH LOGIT LABEL...")
logit_dataset = LogitDataset()
with torch.no_grad():
    for images, labels in train_loader:
        logit_image = images.to(device)
        labels = labels.to(device)
        output1 = teacher_model(logit_image)
        
        logit_soft_label = output1.data
        logit_hard_label = labels.data
        
        logit_dataset.add(logit_image, logit_soft_label, logit_hard_label)
    
    logit_dataset.stackAll()

print ("TRAINING ENHANCED STUDENT WITH SOFTMAX LABEL...")
train_loader_logit = torch.utils.data.DataLoader(dataset=logit_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

# Loss and optimizer for student enhanced
criterion_student_enhanced = cross_entropy
optimizer_student_enhanced = torch.optim.Adam(student_enhanced_model.parameters(), lr=learning_rate) 

total_step = len(train_loader_logit)
for epoch in range(num_epochs):
    for i, (images, soft_labels, hard_labels) in enumerate(train_loader_logit):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        soft_labels = soft_labels.to(device)
        hard_labels = hard_labels.to(device)
        
        outputs_student_enhanced = student_enhanced_model(images)

        # Apply Softmax with Temperature
        softmax = nn.Softmax()
        logSoftmax = nn.LogSoftmax()
        
        softmax_outputs_teacher = softmax(torch.div(soft_labels, train_temp))
        log_softmax_output_student = logSoftmax(torch.div(outputs_student_enhanced, train_temp))

        loss_student_enhanced = criterion_student_enhanced (log_softmax_output_student, softmax_outputs_teacher, outputs_student_enhanced, hard_labels, weight_ratio)

        # Backward and optimize
        optimizer_student_enhanced.zero_grad()
        loss_student_enhanced.backward()
        optimizer_student_enhanced.step()
        
        if (i+1) % 600 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss Enhanced Student: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss_student_enhanced.item()))

# Test the model
student_enhanced_model = student_enhanced_model.eval()

with torch.no_grad():
    correct1 = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        output1 = student_enhanced_model(images)
        _, predicted1 = torch.max(output1.data, 1)
        total += labels.size(0)
        correct1 += (predicted1 == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: STUDENT ENHANCED: {} %'.format(100 * correct1 / total))

Epoch [1/100], Step [600/600], Loss Teacher: 0.0606, Loss Student: 0.2553
Epoch [2/100], Step [600/600], Loss Teacher: 0.0158, Loss Student: 0.1775
Epoch [3/100], Step [600/600], Loss Teacher: 0.0431, Loss Student: 0.1917
Epoch [4/100], Step [600/600], Loss Teacher: 0.0286, Loss Student: 0.2152
Epoch [5/100], Step [600/600], Loss Teacher: 0.0891, Loss Student: 0.2553
Epoch [6/100], Step [600/600], Loss Teacher: 0.0341, Loss Student: 0.1825
Epoch [7/100], Step [600/600], Loss Teacher: 0.0238, Loss Student: 0.0862
Epoch [8/100], Step [600/600], Loss Teacher: 0.0483, Loss Student: 0.1951
Epoch [9/100], Step [600/600], Loss Teacher: 0.0047, Loss Student: 0.1757
Epoch [10/100], Step [600/600], Loss Teacher: 0.0046, Loss Student: 0.0908
Epoch [11/100], Step [600/600], Loss Teacher: 0.0286, Loss Student: 0.1363
Epoch [12/100], Step [600/600], Loss Teacher: 0.0707, Loss Student: 0.1580
Epoch [13/100], Step [600/600], Loss Teacher: 0.0092, Loss Student: 0.1551
Epoch [14/100], Step [600/600], Lo

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/100], Step [600/600], Loss Enhanced Student: 0.6191
Epoch [2/100], Step [600/600], Loss Enhanced Student: 0.4313
Epoch [3/100], Step [600/600], Loss Enhanced Student: 0.6131
Epoch [4/100], Step [600/600], Loss Enhanced Student: 0.4827
Epoch [5/100], Step [600/600], Loss Enhanced Student: 0.4700
Epoch [6/100], Step [600/600], Loss Enhanced Student: 0.5517
Epoch [7/100], Step [600/600], Loss Enhanced Student: 0.4702
Epoch [8/100], Step [600/600], Loss Enhanced Student: 0.3896
Epoch [9/100], Step [600/600], Loss Enhanced Student: 0.3924
Epoch [10/100], Step [600/600], Loss Enhanced Student: 0.4468
Epoch [11/100], Step [600/600], Loss Enhanced Student: 0.5708
Epoch [12/100], Step [600/600], Loss Enhanced Student: 0.5075
Epoch [13/100], Step [600/600], Loss Enhanced Student: 0.4797
Epoch [14/100], Step [600/600], Loss Enhanced Student: 0.3302
Epoch [15/100], Step [600/600], Loss Enhanced Student: 0.4234
Epoch [16/100], Step [600/600], Loss Enhanced Student: 0.3497
Epoch [17/100], S